In [35]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to show the raw code."></form>''')

In [36]:
#increasing cell width with Jupyter notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [37]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
import bqplot as bq

In [38]:
#read data and create dataframe
df = pd.read_csv("eda_final.csv", index_col=0)
df.head()

,age,gender,height,weight,systolic,diastolic,cholesterol,gluc,smoke,alco,active,cardio,bmi,HTN_stage,age_range
0,50.391781,2,168,62.0,110.0,80.0,1,1,0,0,1,0,21.967120,HBP_stage1,50.0
1,55.419178,1,156,85.0,140.0,90.0,3,1,0,0,1,1,34.927679,HBP_stage2,50.0
2,51.663014,1,165,64.0,130.0,70.0,3,1,0,0,0,1,23.507805,HBP_stage1,50.0
3,48.282192,2,169,82.0,150.0,100.0,1,1,0,0,1,1,28.710479,HBP_stage2,40.0
4,47.873973,1,156,56.0,100.0,60.0,1,1,0,0,0,0,23.011177,Normal,40.0


In [ ]:
#create function to categorize blood pressure
def blood_pressure_cat(diastolic):
    if diastolic >= 120:
        return "Crisis"
    elif (90 <= diastolic < 120):
        return "HBP_stage2"
    elif (80 <= diastolic < 90):
        return "HBP_stage1"
    else:
        return "Elevated/Normal"

In [ ]:
#run category function on dataframe
df['HTN_stage'] = df['diastolic'].apply(lambda x: blood_pressure_cat(x))

In [39]:
#create function to categorize age
def age_category(age):
    if age < 40:
        return "30-39"
    elif (50 > age >= 40):
        return "40-49"
    elif (60 > age >= 50):
        return "50-59"
    else:
        return "60-69"
#run category function on dataframe
df['age_range'] = df['age'].apply(lambda x: age_category(x))

In [40]:
#creating handlers for data attibutes
gender_drop= widgets.Dropdown(options=[("Yes",1),
                                       ("No",0)],description="Include gender:",
                                         style= {'description_width': 'initial'})
run_button = widgets.Button(description="Show statistics",
                           button_style='success')

In [41]:
#function to show first plot
def show_plot_gender(b=None):
    gen=gender_drop.value
    plt.figure(figsize= (6,6))
    if gen == 0:
        ax =sns.countplot(y= 'HTN_stage', hue='gender', data= df, 
                          order=["Elevated/Normal","HBP_stage1", "HBP_stage2", "Crisis"])
        plt.title('Diastolic Blood Pressure Stages By Gender')
        plt.legend(["female", "male"])
    else:
        ax =sns.countplot(y= 'HTN_stage', data= df, 
                          order=["Elevated/Normal","HBP_stage1", "HBP_stage2", "Crisis"])
        plt.title('Diastolic Blood Pressure Stages Overview')   
    plt.ylabel('HTN stage')
    total = len(df['age_range'])
    for p in ax.patches:
        percentage = '{:.1f}%'.format(100 * p.get_width()/total)
        x = p.get_x() + p.get_width() + 0.02
        y = p.get_y() + p.get_height()/2
        ax.annotate(percentage, (x, y))

#function to show second plot
def show_plot_age(b=None):
    plt.figure(figsize= (6,6))
    sns.countplot(x= 'age_range', hue= 'HTN_stage', data= df, order=["30-39","40-49","50-59","60-69"])
    plt.title('Diastolic Blood Pressure Categories For Age Ranges', fontweight= 'bold', fontsize= 12)
    plt.xlabel('Age ranges')
    plt.ylabel('Number of Patients')
    plt.legend(["Elevated/Normal","HBP_stage1", "HBP_stage2", "Crisis"])

#function to show third plot    
def show_plot_cardio(b=None):
    plt.figure(figsize= (6,6))
    sns.countplot(x= 'HTN_stage', hue= 'cardio', 
                  data= df, order=["Elevated/Normal","HBP_stage1", "HBP_stage2", "Crisis"],
                 palette="Set1")
    plt.title('Diastolic Blood Pressure Categories With Cardio Disease Coexistance', fontweight= 'bold', fontsize= 12)
    plt.xlabel('Diastolic Blood Pressure Categories')
    plt.ylabel('Number of Patients')
    plt.legend(["cardio disease", "no cardio disease"])

In [43]:
#tabs

def plot_on_click(b):
    out.clear_output(wait=True)
    with out:
        show_plot_gender()
        show_plot_age()
        show_plot_cardio()
        plt.show()

run_button.on_click(plot_on_click)
out=widgets.Output()
widget_box=widgets.VBox([gender_drop,run_button])

tab0 = widgets.Text('Age')
tab1 = widgets.Text('BMI')
tab2 = widgets.Text('Blood Pressure')
tab3 = widgets.Text('Systolic BP')
tab4 = widgets.HBox([widget_box,out])


tab_contents = ['Age', 'BMI', 'Blood Pressure', 'Systolic Blood Pressure', 'Diastolic Blood Pressure']
children = [tab0, tab1, tab2, tab3, tab4]
tab = widgets.Tab()
tab.children = children
for i in range(len(tab_contents)):
    tab.set_title(i, tab_contents[i])
tab